In [1]:
import openai
import json
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
todo_list = []

In [2]:
def add_task(task_description):
    """'할 일 목록(To-Do List)'에 새로운 할 일을 추가합니다."""
    print(f"   (🛠️ 도구 실행: '{task_description}' 항목을 메모장에 기록합니다.)")
    todo_list.append(task_description)
    return json.dumps({"status": "성공"})

In [3]:
def view_tasks():
    """'할 일 목록'에 있는 모든 항목을 보여줍니다."""
    print("   (🛠️ 도구 실행: 메모장의 모든 할 일을 확인합니다.)")
    return json.dumps({"tasks": todo_list})

In [4]:
# AI에게 제출할 '도구 등록 신청서' 목록을 준비합니다.
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_task",
            "description": "할 일 목록(To-Do List)에 새로운 할 일을 추가합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_description": {
                        "type": "string",
                        "description": "추가할 할 일의 내용",
                    }
                },
                "required": ["task_description"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "view_tasks",
            "description": "할 일 목록에 있는 모든 항목을 보여줍니다.",
            "parameters": {"type": "object", "properties": {}},
        },
    },
]

In [5]:
def process_conversation_turn(messages, user_input):
    print(f"😎 사용자: {user_input}")
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-4o", messages=messages, tools=tools, tool_choice="auto"
    )
    response_message = response.choices[0].message
    messages.append(response_message)

    if response_message.tool_calls:
        # 사용 가능한 도구들을 이름으로 쉽게 찾아 쓸 수 있도록 딕셔너리로 만듭니다.
        available_functions = {"add_task": add_task, "view_tasks": view_tasks}

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            # 실제 도구(함수)를 실행하고 결과를 받습니다.
            function_response = function_to_call(**function_args)

            # **[가장 중요!]** 도구 실행 결과를 API 규칙에 맞게 대화 기록부에 추가합니다.
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

            # 3. 도구 실행 후, 최종적으로 사용자에게 보여줄 자연스러운 답변을 AI에게 다시 요청합니다.
            final_response = client.chat.completions.create(
                model="gpt-4o", 
                messages=messages
            )
            print(f"🎉 AI 비서: {final_response.choices[0].message.content}\n")
    else:
        # 도구를 사용하지 않았다면 AI의 첫 응답을 그대로 출력합니다.
        print(f"🎉 AI 비서: {response_message.content}\n")

    return messages

In [6]:
messages = [{"role": "system", "content": "너는 유능한 할 일 관리 비서야. 사용자의 요청에 따라 할 일을 추가하거나 목록을 보여줘. 모든 작업 후에는 사용자에게 친절하게 완료 사실을 알려줘."}]
print("--- 대화 시작 ---\n")

# 첫 번째 대화: '팀 회의' 추가
messages = process_conversation_turn(messages, "내일 아침 9시 팀 회의 일정 추가해줘.")


--- 대화 시작 ---

😎 사용자: 내일 아침 9시 팀 회의 일정 추가해줘.
   (🛠️ 도구 실행: '내일 아침 9시 팀 회의' 항목을 메모장에 기록합니다.)
🎉 AI 비서: 팀 회의 일정이 내일 아침 9시로 추가되었습니다. 다른 도움이 필요하시면 말씀해 주세요!



In [7]:
# 두 번째 대화: '책 반납' 추가
messages = process_conversation_turn(messages, "그리고 오후에 책 반납하는 것도 잊지 말고.")

😎 사용자: 그리고 오후에 책 반납하는 것도 잊지 말고.
   (🛠️ 도구 실행: '오후에 책 반납' 항목을 메모장에 기록합니다.)
🎉 AI 비서: 팀 회의 일정과 책 반납 일정이 성공적으로 추가되었습니다. 다른 할 일이 필요하시면 언제든지 말씀해 주세요!



In [8]:
# 세 번째 대화: 할 일 목록 조회
messages = process_conversation_turn(messages, "지금까지 할 일 목록 좀 보여줘.")

😎 사용자: 지금까지 할 일 목록 좀 보여줘.
   (🛠️ 도구 실행: 메모장의 모든 할 일을 확인합니다.)
🎉 AI 비서: 현재 할 일 목록은 다음과 같습니다:

1. 내일 아침 9시 팀 회의
2. 오후에 책 반납

도움이 필요하면 언제든지 말씀해 주세요!



In [9]:
# --- 최종 결과 확인 ---
print("--- 대화 종료 후 최종 메모장 상태 ---")
print(todo_list)

--- 대화 종료 후 최종 메모장 상태 ---
['내일 아침 9시 팀 회의', '오후에 책 반납']
